# Importing required libraries

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

#Folium install
!pip install folium
import folium

print('Libraries imported.')

     |████████████████████████████████| 102kB 10.4MB/s ta 0:00:01
Libraries imported.


# Downloading New York dataset

In [3]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


# Converting the dataset to Pandas data frame

In [4]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [5]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


# Fetching NY's neighborhoods coordinates (latitude and longitude) 

In [6]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [7]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


# Map creation using folium

In [8]:
import folium
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

# Working with data imported through Foursquare's API 

In [9]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [10]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'LI2QLX1ILGOQRQ2B5UOYVM1U4AZB5GFBG1UDEALLC4EN3B4R'
CLIENT_SECRET = 'QGIAV3NMF0RQKWFGZGVLKNHP1DR5XX4GONSQYJB5EXLRF2G1'
VERSION = '20181020'

In [11]:
#https://developer.foursquare.com/docs/resources/categories
#Coffe Shop = 4bf58dd8d48988d1e0931735
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_coffeshop = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1e0931735')
newyork_venues_coffeshop.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
1,Marble Hill,40.876551,-73.91066,Starbucks,40.873755,-73.908613,Coffee Shop
2,Marble Hill,40.876551,-73.91066,Starbucks,40.873234,-73.908730,Coffee Shop
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Buuni Coffee,40.886862,-73.907016,Coffee Shop


In [12]:
newyork_venues_coffeshop.shape

(1787, 7)

In [13]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

# Map representing all coffe shops in Manhattan 

In [15]:
map_newyork_coffeshop = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_coffeshop, 'red', map_newyork_coffeshop)

map_newyork_coffeshop

In [16]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [17]:
manhattan_grouped = newyork_venues_coffeshop.groupby('Neighborhood').count()
manhattan_grouped
#print('There are {} uniques categories.'.format(len(newyork_venues_coffeshop['Venue Category'].unique())))

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,50,50,50,50,50,50
Carnegie Hill,49,49,49,49,49,49
Central Harlem,32,32,32,32,32,32
Chelsea,50,50,50,50,50,50
Chinatown,50,50,50,50,50,50
Civic Center,50,50,50,50,50,50
Clinton,50,50,50,50,50,50
East Harlem,39,39,39,39,39,39
East Village,50,50,50,50,50,50


# Neighbrohood analysis and clusters

In [18]:
# one hot encoding
manhattan_onehot = pd.get_dummies(newyork_venues_coffeshop[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = newyork_venues_coffeshop['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,American Restaurant,Bagel Shop,Bakery,Bar,Bookstore,Bubble Tea Shop,Café,Clothing Store,Coffee Shop,Convenience Store,Creperie,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Donut Shop,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Gourmet Shop,Hobby Shop,Ice Cream Shop,Italian Restaurant,Juice Bar,Mediterranean Restaurant,New American Restaurant,Pizza Place,Restaurant,Sandwich Place,Soup Place,Spa,Taco Place,Tea Room
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Marble Hill,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,American Restaurant,Bagel Shop,Bakery,Bar,Bookstore,Bubble Tea Shop,Café,Clothing Store,Coffee Shop,Convenience Store,Creperie,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Donut Shop,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Gourmet Shop,Hobby Shop,Ice Cream Shop,Italian Restaurant,Juice Bar,Mediterranean Restaurant,New American Restaurant,Pizza Place,Restaurant,Sandwich Place,Soup Place,Spa,Taco Place,Tea Room
0,Battery Park City,0.00,0.000000,0.020000,0.00000,0.00,0.00,0.020000,0.00,0.820000,0.00,0.000000,0.00000,0.000000,0.00000,0.000000,0.060000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.04,0.00,0.0000,0.000000,0.0200,0.020000,0.00,0.00,0.000000,0.00
1,Carnegie Hill,0.00,0.020408,0.040816,0.00000,0.00,0.00,0.040816,0.00,0.653061,0.00,0.000000,0.00000,0.000000,0.00000,0.000000,0.204082,0.00,0.000000,0.020408,0.00000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.0000,0.000000,0.0000,0.000000,0.00,0.00,0.020408,0.00
2,Central Harlem,0.00,0.000000,0.000000,0.00000,0.00,0.00,0.031250,0.00,0.656250,0.00,0.000000,0.00000,0.000000,0.00000,0.000000,0.312500,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.0000,0.000000,0.0000,0.000000,0.00,0.00,0.000000,0.00
3,Chelsea,0.00,0.000000,0.020000,0.00000,0.00,0.00,0.140000,0.00,0.760000,0.00,0.000000,0.00000,0.000000,0.02000,0.000000,0.020000,0.02,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.0000,0.000000,0.0200,0.000000,0.00,0.00,0.000000,0.00
4,Chinatown,0.00,0.000000,0.000000,0.00000,0.02,0.00,0.000000,0.02,0.780000,0.00,0.000000,0.00000,0.000000,0.00000,0.000000,0.060000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.06,0.02,0.0000,0.000000,0.0000,0.000000,0.02,0.00,0.000000,0.02
5,Civic Center,0.00,0.000000,0.020000,0.00000,0.02,0.00,0.040000,0.02,0.760000,0.00,0.000000,0.00000,0.000000,0.00000,0.000000,0.040000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.02,0.02,0.0000,0.000000,0.0200,0.020000,0.00,0.00,0.000000,0.02
6,Clinton,0.00,0.000000,0.020000,0.00000,0.00,0.02,0.040000,0.00,0.840000,0.00,0.000000,0.00000,0.020000,0.00000,0.000000,0.040000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.0000,0.020000,0.0000,0.000000,0.00,0.00,0.000000,0.00
7,East Harlem,0.00,0.000000,0.025641,0.00000,0.00,0.00,0.076923,0.00,0.589744,0.00,0.025641,0.00000,0.000000,0.00000,0.000000,0.230769,0.00,0.000000,0.025641,0.00000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.0000,0.000000,0.0000,0.000000,0.00,0.00,0.025641,0.00
8,East Village,0.00,0.000000,0.020000,0.02000,0.00,0.00,0.080000,0.00,0.720000,0.00,0.000000,0.00000,0.000000,0.04000,0.000000,0.080000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.00,0.02,0.02,0.00,0.0000,0.000000,0.0000,0.000000,0.00,0.00,0.000000,0.00
9,Financial District,0.00,0.000000,0.020000,0.00000,0.02,0.00,0.020000,0.00,0.800000,0.00,0.000000,0.00000,0.000000,0.00000,0.000000,0.060000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.04,0.00,0.0000,0.000000,0.0200,0.020000,0.00,0.00,0.000000,0.00


In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Coffee Shop,Donut Shop,Juice Bar,Bakery,Sandwich Place,Restaurant,Café,Dessert Shop,Deli / Bodega,Cupcake Shop
1,Carnegie Hill,Coffee Shop,Donut Shop,Café,Bakery,Food Court,Taco Place,Bagel Shop,Bubble Tea Shop,Bookstore,Clothing Store
2,Central Harlem,Coffee Shop,Donut Shop,Café,Convenience Store,Diner,Dessert Shop,Deli / Bodega,Cupcake Shop,Creperie,Tea Room
3,Chelsea,Coffee Shop,Café,Donut Shop,Bakery,Dessert Shop,Restaurant,Flower Shop,Creperie,Diner,Deli / Bodega
4,Chinatown,Coffee Shop,Juice Bar,Donut Shop,Bookstore,Clothing Store,Mediterranean Restaurant,Tea Room,Soup Place,Bubble Tea Shop,Bar


In [22]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 1, 2, 0, 0, 0, 1, 2, 0], dtype=int32)

In [23]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Coffee Shop,Donut Shop,Taco Place,Diner,Dessert Shop,Deli / Bodega,Cupcake Shop,Creperie,Convenience Store,Tea Room
1,Manhattan,Chinatown,40.715618,-73.994279,0,Coffee Shop,Juice Bar,Donut Shop,Bookstore,Clothing Store,Mediterranean Restaurant,Tea Room,Soup Place,Bubble Tea Shop,Bar
2,Manhattan,Washington Heights,40.851903,-73.936900,4,Donut Shop,Coffee Shop,Restaurant,New American Restaurant,Café,Bar,Bookstore,Bakery,Bubble Tea Shop,Flower Shop
3,Manhattan,Inwood,40.867684,-73.921210,3,Coffee Shop,Donut Shop,Bakery,Diner,Convenience Store,Dessert Shop,Deli / Bodega,Cupcake Shop,Creperie,Tea Room
4,Manhattan,Hamilton Heights,40.823604,-73.949688,1,Coffee Shop,Donut Shop,Bakery,Café,Convenience Store,Diner,Dessert Shop,Deli / Bodega,Cupcake Shop,Creperie


In [24]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [25]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Coffee Shop,Juice Bar,Donut Shop,Bookstore,Clothing Store,Mediterranean Restaurant,Tea Room,Soup Place,Bubble Tea Shop,Bar
5,Manhattanville,Coffee Shop,Donut Shop,Café,French Restaurant,Bagel Shop,Bakery,Bar,Bookstore,Bubble Tea Shop,Clothing Store
14,Clinton,Coffee Shop,Donut Shop,Café,Bakery,Deli / Bodega,Pizza Place,Bubble Tea Shop,Creperie,Diner,Dessert Shop
15,Midtown,Coffee Shop,Donut Shop,Juice Bar,Bakery,Café,Convenience Store,Dessert Shop,American Restaurant,Bar,Bookstore
16,Murray Hill,Coffee Shop,Donut Shop,Café,Dessert Shop,American Restaurant,Bakery,Juice Bar,Bookstore,Bubble Tea Shop,Bar
22,Little Italy,Coffee Shop,Donut Shop,Café,Clothing Store,Juice Bar,Bookstore,Mediterranean Restaurant,Tea Room,Bubble Tea Shop,Bar
26,Morningside Heights,Coffee Shop,Donut Shop,Bakery,French Restaurant,Bagel Shop,Bar,Bookstore,Bubble Tea Shop,Café,Clothing Store
28,Battery Park City,Coffee Shop,Donut Shop,Juice Bar,Bakery,Sandwich Place,Restaurant,Café,Dessert Shop,Deli / Bodega,Cupcake Shop
29,Financial District,Coffee Shop,Donut Shop,Juice Bar,Bakery,Sandwich Place,Restaurant,Bookstore,Café,Convenience Store,Dessert Shop
31,Noho,Coffee Shop,Donut Shop,Café,Juice Bar,Bookstore,Bakery,Mediterranean Restaurant,Clothing Store,Tea Room,Bar


In [26]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Coffee Shop,Donut Shop,Taco Place,Diner,Dessert Shop,Deli / Bodega,Cupcake Shop,Creperie,Convenience Store,Tea Room
4,Hamilton Heights,Coffee Shop,Donut Shop,Bakery,Café,Convenience Store,Diner,Dessert Shop,Deli / Bodega,Cupcake Shop,Creperie
6,Central Harlem,Coffee Shop,Donut Shop,Café,Convenience Store,Diner,Dessert Shop,Deli / Bodega,Cupcake Shop,Creperie,Tea Room
7,East Harlem,Coffee Shop,Donut Shop,Café,Food Court,Creperie,Taco Place,Bakery,Bubble Tea Shop,Bookstore,Clothing Store
9,Yorkville,Coffee Shop,Donut Shop,Café,Bakery,Bagel Shop,Hobby Shop,Bar,Bookstore,Bubble Tea Shop,Clothing Store
11,Roosevelt Island,Coffee Shop,Donut Shop,Sandwich Place,Café,Hobby Shop,Convenience Store,Diner,Dessert Shop,Deli / Bodega,Cupcake Shop
25,Manhattan Valley,Coffee Shop,Donut Shop,Food Truck,Café,Bakery,Food Stand,Pizza Place,Tea Room,Creperie,Dessert Shop
30,Carnegie Hill,Coffee Shop,Donut Shop,Café,Bakery,Food Court,Taco Place,Bagel Shop,Bubble Tea Shop,Bookstore,Clothing Store
36,Tudor City,Coffee Shop,Donut Shop,Café,Convenience Store,American Restaurant,Sandwich Place,Juice Bar,Bar,Bookstore,Bubble Tea Shop
37,Stuyvesant Town,Coffee Shop,Donut Shop,Café,Bagel Shop,Bakery,Bar,Cupcake Shop,Dessert Shop,Food Truck,Bookstore


In [27]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Upper East Side,Coffee Shop,Café,Donut Shop,Bakery,Diner,Juice Bar,Bagel Shop,Bubble Tea Shop,Bookstore,Clothing Store
10,Lenox Hill,Coffee Shop,Café,Donut Shop,Juice Bar,Bakery,Sandwich Place,Bagel Shop,Diner,Bar,Bookstore
13,Lincoln Square,Coffee Shop,Café,Donut Shop,Deli / Bodega,Bakery,Juice Bar,Bubble Tea Shop,Diner,Pizza Place,Sandwich Place
17,Chelsea,Coffee Shop,Café,Donut Shop,Bakery,Dessert Shop,Restaurant,Flower Shop,Creperie,Diner,Deli / Bodega
18,Greenwich Village,Coffee Shop,Café,Donut Shop,Dessert Shop,Clothing Store,Mediterranean Restaurant,Juice Bar,Bookstore,Bubble Tea Shop,Flower Shop
19,East Village,Coffee Shop,Donut Shop,Café,Dessert Shop,Italian Restaurant,Juice Bar,Bar,Bakery,Bubble Tea Shop,Bookstore
20,Lower East Side,Coffee Shop,Donut Shop,Café,Juice Bar,Dessert Shop,Ice Cream Shop,Tea Room,Bar,Bakery,Spa
21,Tribeca,Coffee Shop,Donut Shop,Café,Clothing Store,Mediterranean Restaurant,Dessert Shop,Gourmet Shop,Juice Bar,Tea Room,Sandwich Place
23,Soho,Coffee Shop,Donut Shop,Clothing Store,Café,Tea Room,Mediterranean Restaurant,Dessert Shop,Gourmet Shop,Juice Bar,Bookstore
24,West Village,Coffee Shop,Café,Donut Shop,Bakery,Bagel Shop,Clothing Store,Flower Shop,Gourmet Shop,Dessert Shop,Bar


In [28]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Inwood,Coffee Shop,Donut Shop,Bakery,Diner,Convenience Store,Dessert Shop,Deli / Bodega,Cupcake Shop,Creperie,Tea Room
12,Upper West Side,Coffee Shop,Donut Shop,Café,Deli / Bodega,Bakery,Food & Drink Shop,Food Truck,French Restaurant,Diner,Pizza Place


In [29]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Donut Shop,Coffee Shop,Restaurant,New American Restaurant,Café,Bar,Bookstore,Bakery,Bubble Tea Shop,Flower Shop


# End of assignment